In [1]:
import numpy as np
import pandas as pd
import os
from scipy.stats.mstats import winsorize
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
import xgboost as xgb
import csv

In [2]:
directory_path=(r"C:\Users\ArtisusXiren\Desktop\predict-energy-behavior-of-prosumers\Data")
csv_dict=os.listdir(r"C:\Users\ArtisusXiren\Desktop\predict-energy-behavior-of-prosumers\Data")
csv_files=[files for files in csv_dict if files.endswith(".csv")]
file_dict={}
for data_file in csv_files:
    file_path=os.path.join(directory_path,data_file)
    file_name=pd.read_csv(file_path)
    null=file_name.isnull().sum().sum()
    categorical=file_name.select_dtypes(include=['object']).columns
    file_dict[data_file]=file_name
    for i in file_name.columns:
        if i in categorical and null>0:
            replace_value=file_name[i].mode()[0]
            file_name[i].fillna(replace_value,inplace=True)
            # print(file_name[i])
        
           
        elif i not in categorical and null>0:
            replace_value=file_name[i].mean()
            file_name[i].fillna(replace_value,inplace=True)
            #print(file_name[i])
            
for name,value in file_dict.items():
    null_type=value.isnull().sum().sum()
    print(name)
    print(null_type)
    
    


client.csv
0
electricity_prices.csv
0
forecast_weather.csv
0
gas_prices.csv
0
historical_weather.csv
0
Sample_submission.csv
0
train.csv
0
weather_station_to_county_mapping.csv
0


In [3]:
outlier_dict={}
for data_file in csv_files:
    file_path=os.path.join(directory_path,data_file)
    file_name=pd.read_csv(file_path)
    for columns in file_name.columns:
        if pd.api.types.is_numeric_dtype(file_name[columns]):
            q1=file_name[columns].quantile(0.25)
            q2=file_name[columns].quantile(0.75)
            iqr=q2-q1        
            lower=q1-1.5*iqr
            upper=q2+1.5*iqr
            outliers=(file_name[columns]< lower)|(file_name[columns] > upper)
            outlier_dict[columns]=outliers
            print(f"Before:")
            print(file_name[columns].describe())
outlier_list={}
for name,value in outlier_dict.items():
    if value.sum()>70:
        outlier_list[name]=value.sum()
outlier_list

    

Before:
count    41919.000000
mean         1.898996
std          1.081713
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          3.000000
Name: product_type, dtype: float64
Before:
count    41919.000000
mean         7.297097
std          4.780750
min          0.000000
25%          3.000000
50%          7.000000
75%         11.000000
max         15.000000
Name: county, dtype: float64
Before:
count    41919.000000
mean        73.345118
std        144.064389
min          5.000000
25%         13.000000
50%         32.000000
75%         70.000000
max       1517.000000
Name: eic_count, dtype: float64
Before:
count    41919.000000
mean      1450.771451
std       2422.233120
min          5.500000
25%        321.900000
50%        645.200000
75%       1567.150000
max      19314.310000
Name: installed_capacity, dtype: float64
Before:
count    41919.000000
mean         0.536773
std          0.498652
min          0.000000
25%          0.000000
50%      

{'eic_count': 4102,
 'installed_capacity': 3487,
 'euros_per_mwh': 962,
 'temperature': 6138,
 'dewpoint': 5446,
 '10_metre_u_wind_component': 41559,
 '10_metre_v_wind_component': 44448,
 'direct_solar_radiation': 309265,
 'surface_solar_radiation_downwards': 435228,
 'snowfall': 147477,
 'total_precipitation': 590740,
 'rain': 245427,
 'surface_pressure': 26534,
 'windspeed_10m': 32772,
 'shortwave_radiation': 210545,
 'diffuse_radiation': 69447,
 'target': 274217}

In [4]:
for data_file in csv_files:
    file_path=os.path.join(directory_path,data_file)
    file_name=pd.read_csv(file_path)
    for columns in file_name.columns:
        if columns in outlier_list:
            win_data=winsorize(file_name[columns],limits=[0.05,0.05])
            file_name[columns]=win_data
        print(f"After:")
        print(file_name[columns].describe())
merged_df=pd.read_csv(r"C:\Users\ArtisusXiren\Desktop\predict-energy-behavior-of-prosumers\Data\train.csv")
new_data_files=[i for i in csv_files if i!="train.csv"]

After:
count    41919.000000
mean         1.898996
std          1.081713
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          3.000000
Name: product_type, dtype: float64
After:
count    41919.000000
mean         7.297097
std          4.780750
min          0.000000
25%          3.000000
50%          7.000000
75%         11.000000
max         15.000000
Name: county, dtype: float64
After:
count    41919.000000
mean        58.295618
std         70.051600
min          6.000000
25%         13.000000
50%         32.000000
75%         70.000000
max        278.000000
Name: eic_count, dtype: float64
After:
count    41919.000000
mean      1210.464276
std       1359.108257
min         90.500000
25%        321.900000
50%        645.200000
75%       1567.150000
max       5523.400000
Name: installed_capacity, dtype: float64
After:
count    41919.000000
mean         0.536773
std          0.498652
min          0.000000
25%          0.000000
50%          1

C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


After:
count    3.424512e+06
mean     5.865000e+01
std      6.873865e-01
min      5.760000e+01
25%      5.812500e+01
50%      5.865000e+01
75%      5.917500e+01
max      5.970000e+01
Name: latitude, dtype: float64
After:
count    3.424512e+06
mean     2.495000e+01
std      2.015565e+00
min      2.170000e+01
25%      2.320000e+01
50%      2.495000e+01
75%      2.670000e+01
max      2.820000e+01
Name: longitude, dtype: float64
After:
count                 3424512
unique                    637
top       2021-09-01 02:00:00
freq                     5376
Name: origin_datetime, dtype: object
After:
count    3.424512e+06
mean     2.450000e+01
std      1.385340e+01
min      1.000000e+00
25%      1.275000e+01
50%      2.450000e+01
75%      3.625000e+01
max      4.800000e+01
Name: hours_ahead, dtype: float64
After:
count    3.424512e+06
mean     5.758409e+00
std      7.213260e+00
min     -5.988867e+00
25%      2.606445e-01
50%      4.872705e+00
75%      1.114639e+01
max      1.951199e+01
Name: t

C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


After:
count    3.424512e+06
mean     2.471358e+00
std      6.580246e+00
min     -8.711890e+00
25%     -2.364355e+00
50%      1.835596e+00
75%      7.302026e+00
max      1.506069e+01
Name: dewpoint, dtype: float64
After:
count    3.424512e+06
mean     3.946654e-01
std      4.440425e-01
min      0.000000e+00
25%      0.000000e+00
50%      8.866882e-02
75%      9.784851e-01
max      1.000008e+00
Name: cloudcover_high, dtype: float64
After:
count    3.424512e+06
mean     4.346453e-01
std      4.386346e-01
min      0.000000e+00
25%      3.356934e-04
50%      2.305453e-01
75%      9.994587e-01
max      1.000008e+00
Name: cloudcover_low, dtype: float64
After:


...

count    1.710802e+06
mean     3.192708e+02
std      1.837298e+02
min      1.000000e+00
25%      1.600000e+02
50%      3.190000e+02
75%      4.780000e+02
max      6.370000e+02
Name: data_block_id, dtype: float64
After:
count    4.509800e+04
mean     2.012112e+06
std      1.895210e+03
min      2.005872e+06
25%      2.012112e+06
50%      2.012112e+06
75%      2.012112e+06
max      2.018351e+06
Name: row_id, dtype: float64
After:
count    45098.000000
mean       635.499956
std          1.118046
min        634.000000
25%        634.250000
50%        635.000000
75%        636.000000
max        637.000000
Name: data_block_id, dtype: float64
After:
count    45098.000000
mean       226.302295
std         89.654791
min         27.569246
25%        255.116353
50%        274.855560
75%        274.855560
max        274.855560
Name: target, dtype: float64


C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


After:
count    2.018352e+06
mean     7.297034e+00
std      4.780990e+00
min      0.000000e+00
25%      3.000000e+00
50%      7.000000e+00
75%      1.100000e+01
max      1.500000e+01
Name: county, dtype: float64
After:
count    2.018352e+06
mean     5.368261e-01
std      4.986421e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: is_business, dtype: float64
After:
count    2.018352e+06
mean     1.898927e+00
std      1.081766e+00
min      0.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      3.000000e+00
max      3.000000e+00
Name: product_type, dtype: float64
After:
count    2.018352e+06
mean     1.727118e+02
std      2.968104e+02
min      0.000000e+00
25%      3.790000e-01
50%      3.116800e+01
75%      1.804840e+02
max      1.083633e+03
Name: target, dtype: float64
After:
count    2018352.0
mean           0.5
std            0.5
min            0.0
25%            0.0
50%            0.5
75%            1.0
m

C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


count    2.018352e+06
mean     1.009176e+06
std      5.826482e+05
min      0.000000e+00
25%      5.045878e+05
50%      1.009176e+06
75%      1.513763e+06
max      2.018351e+06
Name: row_id, dtype: float64
After:
count    2.018352e+06
mean     3.304538e+01
std      1.959059e+01
min      0.000000e+00
25%      1.600000e+01
50%      3.300000e+01
75%      5.000000e+01
max      6.800000e+01
Name: prediction_unit_id, dtype: float64
After:
count           49
unique          15
top       Harjumaa
freq             6
Name: county_name, dtype: object
After:
count    112.000000
mean      24.950000
std        2.024623
min       21.700000
25%       23.200000
50%       24.950000
75%       26.700000
max       28.200000
Name: longitude, dtype: float64
After:
count    112.000000
mean      58.650000
std        0.690476
min       57.600000
25%       58.125000
50%       58.650000
75%       59.175000
max       59.700000
Name: latitude, dtype: float64
After:
count    49.000000
mean      7.061224
std       4.8

In [6]:
for data_file in new_data_files:
    file_path=os.path.join(directory_path,data_file)
    file_name=pd.read_csv(file_path)
    merged_df=pd.concat([merged_df,file_name],axis=0,ignore_index=True)


In [7]:
null=merged_df.isnull()
null=null.sum()
categorical=merged_df.select_dtypes(include=['object']).columns
print(null)
print(categorical)

county                               5196398
is_business                          5196447
product_type                         5196447
target                               5193796
is_consumption                       5238366
datetime                             3527564
data_block_id                            112
row_id                               5193268
prediction_unit_id                   5238366
eic_count                            7214799
installed_capacity                   7214799
date                                 7214799
forecast_date                        7240795
euros_per_mwh                        7241432
origin_date                          7240795
latitude                             2121292
longitude                            2121292
origin_datetime                      3832206
hours_ahead                          3832206
temperature                          2121404
dewpoint                             2121404
cloudcover_high                      2121404
cloudcover

In [8]:
for columns in merged_df.columns:
    if columns not in categorical:
        win_data=merged_df[columns].mean()
        merged_df[columns].fillna(win_data,inplace=True)
for columns in categorical:
    merged_df=merged_df.drop([columns],axis=1)
new_null=merged_df.isnull()
new_null=new_null.sum()
new_null

county                               0
is_business                          0
product_type                         0
target                               0
is_consumption                       0
data_block_id                        0
row_id                               0
prediction_unit_id                   0
eic_count                            0
installed_capacity                   0
euros_per_mwh                        0
latitude                             0
longitude                            0
hours_ahead                          0
temperature                          0
dewpoint                             0
cloudcover_high                      0
cloudcover_low                       0
cloudcover_mid                       0
cloudcover_total                     0
10_metre_u_wind_component            0
10_metre_v_wind_component            0
direct_solar_radiation               0
surface_solar_radiation_downwards    0
snowfall                             0
total_precipitation      

In [10]:
q1=merged_df.quantile(0.25)
q2=merged_df.quantile(0.75)
iqr=q2-q1 
def identify(columns):
    lower=q1[columns]-1.5*iqr[columns]
    upper=q2[columns]+1.5*iqr[columns]
    outliers=(merged_df[columns]< lower)|(merged_df[columns] > upper)
    return outliers
outlier_dict={i:identify(i) for i in merged_df.columns}
outlier_list={}
for name,value in outlier_dict.items():
     if value.sum()>100:
        outlier_list[name]=value.sum()
outlier_list
for columns in merged_df.columns:
    if columns in outlier_list:
        win_data=winsorize(merged_df[columns],limits=[0.05,0.05])
        merged_df[columns]=win_data
                        

C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


{'county': 2060320,
 'is_business': 2060271,
 'product_type': 2060271,
 'target': 2062922,
 'is_consumption': 2018352,
 'row_id': 2063450,
 'prediction_unit_id': 2018352,
 'hours_ahead': 3424512,
 'dewpoint': 427821,
 'cloudcover_high': 712132,
 'cloudcover_low': 853294,
 'cloudcover_mid': 717815,
 'cloudcover_total': 1027461,
 '10_metre_u_wind_component': 3424512,
 '10_metre_v_wind_component': 3424512,
 'direct_solar_radiation': 879092,
 'surface_solar_radiation_downwards': 584042,
 'rain': 1465375,
 'surface_pressure': 1710802,
 'windspeed_10m': 1710802,
 'winddirection_10m': 1710802,
 'shortwave_radiation': 1710802,
 'diffuse_radiation': 1710802}

In [10]:
attributes=[columns for columns in merged_df if columns!="target" and columns!="data_block_id" and columns!="row_id" and columns!="prediction_unit_id"]
X=merged_df[attributes].values
y=merged_df['target'].values
f_2,z=f_regression(X,y,center=True,force_finite=True)
results=pd.DataFrame({'Attributes':attributes,'P-value':f_2,'z-value':z})
results

C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:331: RuntimeWarning: invalid value encountered in sqrt
  X_norms = np.sqrt(row_norms(X.T, squared=True) - n_samples * X_means**2)


,Attributes,P-value,z-value
0,county,4.865961e+02,7.907504e-108
1,is_business,1.709236e+05,0.000000e+00
2,product_type,4.719220e+04,0.000000e+00
3,is_consumption,6.171110e+05,0.000000e+00
4,eic_count,0.000000e+00,1.000000e+00
5,installed_capacity,0.000000e+00,1.000000e+00
6,euros_per_mwh,0.000000e+00,1.000000e+00
7,latitude,2.257872e-12,1.000000e+00
8,longitude,1.121299e-13,1.000000e+00
9,hours_ahead,5.914140e-16,1.000000e+00


In [11]:
feature_dict={}
dispersion_dict={}
for columns in merged_df.columns:
    if columns in attributes:
        arr=np.array(merged_df[columns])
        mean_value=np.mean(arr)
        mad_value = np.mean(np.abs(arr-mean_value))
        feature_dict[columns]=mad_value
feature_dict


{'county': 1.0637355723806095,
 'is_business': 0.14206900943487044,
 'product_type': 0.28330012223566137,
 'is_consumption': 0.13993748977344897,
 'eic_count': 1.7479351299698465e-12,
 'installed_capacity': 3.387867764104158e-11,
 'euros_per_mwh': 1.6484591469634324e-12,
 'latitude': 0.4272627370825455,
 'longitude': 1.2461824804966875,
 'hours_ahead': 5.401535854634604,
 'temperature': 4.294515368977657,
 'dewpoint': 3.8274382803785234,
 'cloudcover_high': 13.642307130210362,
 'cloudcover_low': 16.77277873520721,
 'cloudcover_mid': 12.29326181539579,
 'cloudcover_total': 20.626873207997473,
 '10_metre_u_wind_component': 1.2876986464335203,
 '10_metre_v_wind_component': 1.3621407330164397,
 'direct_solar_radiation': 114.51524629280188,
 'surface_solar_radiation_downwards': 61.99346516116252,
 'snowfall': 0.0022806524603162575,
 'total_precipitation': 3.8961328042841794e-05,
 'lowest_price_per_mwh': 1.9895196601282805e-13,
 'highest_price_per_mwh': 1.0516032489249483e-12,
 'rain': 0.016

In [12]:
correlation=np.corrcoef(X.T,y)[:,-1]
for i, coef in zip(attributes,correlation):
    print(f"Correlation coefficient between feature {i} and target: {coef}")

Correlation coefficient between feature county and target: -0.008213973965698312
Correlation coefficient between feature is_business and target: 0.15215916013705483
Correlation coefficient between feature product_type and target: 0.08063106515255741
Correlation coefficient between feature is_consumption and target: 0.28076051427342924
Correlation coefficient between feature eic_count and target: -3.6599004124645795e-12
Correlation coefficient between feature installed_capacity and target: 2.176499093041174e-11
Correlation coefficient between feature euros_per_mwh and target: -1.9885666790863523e-11
Correlation coefficient between feature latitude and target: 1.6069488036476355e-13
Correlation coefficient between feature longitude and target: -8.93110828879152e-14
Correlation coefficient between feature hours_ahead and target: 7.761659497624121e-21
Correlation coefficient between feature temperature and target: 0.0005882696535391354
Correlation coefficient between feature dewpoint and t

In [13]:
for name,value in enumerate(attributes):
    feature_value=X[:,name]
    if np.std (feature_value)== 0:
        print(f"Feature name: {name}, Correlation: {correlation}, P-value: {p_value}")
    else:    
        correlation,p_value=pearsonr(feature_value,y)
        print(f"Feature name: {name}, Correlation: {correlation}, P-value: {p_value}")

Feature name: 0, Correlation: -0.008213973965681623, P-value: 7.90746801603262e-108
Feature name: 1, Correlation: 0.15215916013876096, P-value: 0.0
Feature name: 2, Correlation: 0.08063106516479274, P-value: 0.0
Feature name: 3, Correlation: 0.28076051427392235, P-value: 0.0


C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Feature name: 4, Correlation: nan, P-value: nan
Feature name: 5, Correlation: nan, P-value: nan
Feature name: 6, Correlation: nan, P-value: nan
Feature name: 7, Correlation: 8.253433811497742e-14, P-value: 0.9999999998229043
Feature name: 8, Correlation: -5.659204251077119e-14, P-value: 0.9999999998787963
Feature name: 9, Correlation: -4.726443845678996e-19, P-value: 1.000000000000001
Feature name: 10, Correlation: 0.000588269653555195, P-value: 0.11416041263973775
Feature name: 11, Correlation: 0.0031344585810722065, P-value: 3.84805668953462e-17
Feature name: 12, Correlation: -0.006374464761348592, P-value: 1.0811300998124638e-65
Feature name: 13, Correlation: -0.0022237145988654727, P-value: 2.3483468757462777e-09
Feature name: 14, Correlation: -0.014996075291480268, P-value: 0.0
Feature name: 15, Correlation: 8.477940571793852e-14, P-value: 0.9999999998182021
Feature name: 16, Correlation: 0.00038977184514442625, P-value: 0.2952328496740865
Feature name: 17, Correlation: 0.00437490

In [14]:
mutual_scores=mutual_info_regression(X,y)
for name,score in zip(attributes,mutual_scores):
    print(f"mutual-dependance between feature {name} and target: {score}")

mutual-dependance between feature county and target: 0.5834423898488783
mutual-dependance between feature is_business and target: 0.582660022931206
mutual-dependance between feature product_type and target: 0.5841491480036591
mutual-dependance between feature is_consumption and target: 0.6715410251616705
mutual-dependance between feature eic_count and target: 0.15191141443703948
mutual-dependance between feature installed_capacity and target: 0.15506207529174354
mutual-dependance between feature euros_per_mwh and target: 0.1751134053930432
mutual-dependance between feature latitude and target: 0.5567291595213479
mutual-dependance between feature longitude and target: 0.5562145449403406
mutual-dependance between feature hours_ahead and target: 0.23062000427995955
mutual-dependance between feature temperature and target: 0.5560226594039914
mutual-dependance between feature dewpoint and target: 0.5556632640866503
mutual-dependance between feature cloudcover_high and target: 0.555472541391

In [15]:
selected_attributes=[name for name,value in zip(attributes,mutual_scores) if value>0.5]
model_X=merged_df[selected_attributes].values
X_train,X_test,y_train,y_test=train_test_split(model_X,y,test_size=0.2,random_state=42)
Model=RandomForestRegressor()

In [16]:
Model.fit(X_train,y_train)
y_pred=Model.predict(X_test)
mae=mean_absolute_error(y_test,y_pred)
mae

16.313184786518722

In [30]:
mse= mean_squared_error(y_test,y_pred)
mse

1927.335383763327

In [87]:
Model_xgb=xgb.XGBRegressor(objective='reg:squarederror',learning_rate=0.01,max_depth=8,colsample_bytree=0.8,subsample=0.8,reg_alpha=1,n_estimators=2000)
Model_xgb.fit(X_train,y_train)
y_pred_xgb=Model_xgb.predict(X_test)


In [88]:
mae_xgb=mean_absolute_error(y_test,y_pred_xgb)
mae_xgb

16.315004963403755

In [73]:
directory_test=(r"C:\Users\ArtisusXiren\Desktop\predict-energy-behavior-of-prosumers\Data\example_test_files")
csv_dir=os.listdir(r"C:\Users\ArtisusXiren\Desktop\predict-energy-behavior-of-prosumers\Data\example_test_files")
csv_test=[files for files in csv_dir if files.endswith(".csv") and files!="sample_submission.csv" and files!="revealed_targets.csv"]
file_dict={}
for data_test in csv_test:
    file_test=os.path.join(directory_test,data_test)
    file_name=pd.read_csv(file_test)
    categorical_test=file_name.select_dtypes(include=['object']).columns
    file_dict[data_test]=file_name
    for columns in file_name.columns:
        null_test=file_name[columns].isnull().sum()
        if columns in categorical_test and null_test>0:
            replace_value=file_name[columns].mode()[0]
            file_name[columns].fillna(replace_value,inplace=True)
        elif columns not in categorical_test and null_test>0:
            replace_value=file_name[columns].mean()
            file_name[columns].fillna(replace_value,inplace=True)
    new_null=file_name.isnull()
    new_null=new_null.sum()
    print(categorical_test)   

Index(['date'], dtype='object')
Index(['forecast_date', 'origin_date'], dtype='object')
Index(['origin_datetime', 'forecast_datetime'], dtype='object')
Index(['forecast_date', 'origin_date'], dtype='object')
Index(['datetime'], dtype='object')
Index(['prediction_datetime'], dtype='object')


In [74]:
csv_test

['client.csv',
 'electricity_prices.csv',
 'forecast_weather.csv',
 'gas_prices.csv',
 'historical_weather.csv',
 'test.csv']

In [75]:
outlier_dict={}    
for data_test in csv_test:
    file_test=os.path.join(directory_test,data_test)
    file_name=pd.read_csv(file_test)
    boolean_column=file_name.select_dtypes(include=['bool']).columns
    file_name[boolean_column]=file_name[boolean_column].astype('int64')
    for columns in file_name.columns:
        if pd.api.types.is_numeric_dtype(file_name[columns]):
            q1=file_name[columns].quantile(0.25)
            q2=file_name[columns].quantile(0.75)
            iqr=q2-q1
            lower=q1-1.5*iqr
            upper=q2+1.5*iqr
            outliers=(file_name[columns]<lower)|(file_name[columns]>upper)
            outlier_dict[columns]=outliers
            print("Before winsorization:")
            print(file_name[columns].describe())    
        
outlier_list_test={}
for name ,value in outlier_dict.items():
    if value.sum()>10:
        outlier_list_test[name]=value.sum()
          


Before winsorization:
count    262.000000
mean       1.877863
std        1.100490
min        0.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        3.000000
Name: product_type, dtype: float64
Before winsorization:
count    262.000000
mean       7.305344
std        4.876864
min        0.000000
25%        3.000000
50%        7.000000
75%       11.000000
max       15.000000
Name: county, dtype: float64
Before winsorization:
count     262.00000
mean      113.79771
std       216.05399
min         5.00000
25%        16.25000
50%        51.00000
75%       101.00000
max      1517.00000
Name: eic_count, dtype: float64
Before winsorization:
count      262.000000
mean      2259.319924
std       3479.433973
min         18.000000
25%        471.175000
50%       1155.430000
75%       2600.600000
max      19314.310000
Name: installed_capacity, dtype: float64
Before winsorization:
count    262.000000
mean       0.549618
std        0.498484
min        0.000000
25%        0.0000

In [76]:
outlier_list_test

{'eic_count': 24,
 'installed_capacity': 20,
 'temperature': 192,
 'dewpoint': 26,
 'cloudcover_high': 1938,
 'cloudcover_low': 1060,
 'cloudcover_total': 328,
 '10_metre_u_wind_component': 260,
 '10_metre_v_wind_component': 167,
 'snowfall': 126,
 'total_precipitation': 3385,
 'rain': 673,
 'surface_pressure': 105,
 'windspeed_10m': 53,
 'winddirection_10m': 368,
 'diffuse_radiation': 18}

In [77]:
for data_test in csv_test:
    file_test=os.path.join(directory_test,data_test)
    file_name=pd.read_csv(file_test)
    for columns in file_name:
        if columns in outlier_list_test:
            print(f"{columns}")
            win_data=winsorize(file_name[columns],limits=[0.05,0.05])
            file_name[columns]=win_data
        print("\nAfter winsorization:")
        print(file_name[columns].describe())


After winsorization:
count    262.000000
mean       1.877863
std        1.100490
min        0.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        3.000000
Name: product_type, dtype: float64

After winsorization:
count    262.000000
mean       7.305344
std        4.876864
min        0.000000
25%        3.000000
50%        7.000000
75%       11.000000
max       15.000000
Name: county, dtype: float64
eic_count

After winsorization:
count    262.000000
mean      94.374046
std      121.616339
min        8.000000
25%       16.250000
50%       51.000000
75%      101.000000
max      482.000000
Name: eic_count, dtype: float64
installed_capacity

After winsorization:
count      262.000000
mean      2027.268473
std       2549.542029
min        136.000000
25%        471.175000
50%       1155.430000
75%       2600.600000
max      10128.200000
Name: installed_capacity, dtype: float64

After winsorization:
count    262.000000
mean       0.549618
std        0.498484
min     

C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(



After winsorization:
count    21504.000000
mean        58.650000
std          0.687402
min         57.600000
25%         58.125000
50%         58.650000
75%         59.175000
max         59.700000
Name: latitude, dtype: float64

After winsorization:
count    21504.000000
mean        24.950000
std          2.015611
min         21.700000
25%         23.200000
50%         24.950000
75%         26.700000
max         28.200000
Name: longitude, dtype: float64

After winsorization:
count                   21504
unique                      4
top       2023-05-27 02:00:00
freq                     5376
Name: origin_datetime, dtype: object

After winsorization:
count    21504.000000
mean        24.500000
std         13.853721
min          1.000000
25%         12.750000
50%         24.500000
75%         36.250000
max         48.000000
Name: hours_ahead, dtype: float64
temperature

After winsorization:
count    21504.000000
mean        11.855221
std          2.831999
min          7.274316
25%     

C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition

count    10752.000000
mean        25.517392
std         24.833267
min          0.000000
25%          3.000000
50%         19.000000
75%         41.000000
max         82.000000
Name: cloudcover_total, dtype: float64
cloudcover_low

After winsorization:
count    10752.000000
mean         6.562593
std         10.259845
min          0.000000
25%          0.000000
50%          1.000000
75%          9.000000
max         37.000000
Name: cloudcover_low, dtype: float64

After winsorization:
count    10752.000000
mean        26.770182
std         29.923741
min          0.000000
25%          0.000000
50%         16.000000
75%         45.000000
max        100.000000
Name: cloudcover_mid, dtype: float64
cloudcover_high

After winsorization:
count    10752.000000
mean        12.867560
std         25.440653
min          0.000000
25%          0.000000
50%          0.000000
75%         10.000000
max         87.000000
Name: cloudcover_high, dtype: float64
windspeed_10m

After winsorization:
count    107

C:\Users\ArtisusXiren\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


In [78]:
test=pd.read_csv(r"C:\Users\ArtisusXiren\Desktop\predict-energy-behavior-of-prosumers\Data\example_test_files\test.csv")
new_test_files=[i for i in csv_test if i!="test.csv"]
for data_test in new_test_files:
    file_test=os.path.join(directory_test,data_test)
    file_name=pd.read_csv(file_test)
    test=pd.concat([test,file_name],axis=0,ignore_index=True)
null=test.isnull()
null=null.sum()
categorical_test=test.select_dtypes(include=['object']).columns
print(null)
print(categorical_test)

county                               32356
is_business                          32356
product_type                         32356
is_consumption                       32618
prediction_datetime                  32618
data_block_id                            0
row_id                               32618
prediction_unit_id                   32618
currently_scored                     32618
eic_count                            44836
installed_capacity                   44836
date                                 44836
forecast_date                        44998
euros_per_mwh                        45002
origin_date                          44998
latitude                             12842
longitude                            12842
origin_datetime                      23594
hours_ahead                          23594
temperature                          12842
dewpoint                             12842
cloudcover_high                      12842
cloudcover_low                       12842
cloudcover_

In [82]:
for columns in test.columns:
    if columns not in categorical_test:
        win_data=test[columns].mean()
        test[columns].fillna(win_data,inplace=True)
#for columns in categorical_test:
    #print(f"{columns}")
    #test_new=test.drop([columns],axis=1)
test_new = test.drop(categorical_test, axis=1)
new_null_test=test_new.isnull()
new_null_test=new_null_test.sum()
new_null_test

county                               0
is_business                          0
product_type                         0
is_consumption                       0
data_block_id                        0
row_id                               0
prediction_unit_id                   0
eic_count                            0
installed_capacity                   0
euros_per_mwh                        0
latitude                             0
longitude                            0
hours_ahead                          0
temperature                          0
dewpoint                             0
cloudcover_high                      0
cloudcover_low                       0
cloudcover_mid                       0
cloudcover_total                     0
10_metre_u_wind_component            0
10_metre_v_wind_component            0
direct_solar_radiation               0
surface_solar_radiation_downwards    0
snowfall                             0
total_precipitation                  0
lowest_price_per_mwh     

In [84]:
q1=test_new.quantile(0.25)
q2=test_new.quantile(0.75)
iqr=q2-q1 
def identify(columns):
    lower=q1[columns]-1.5*iqr[columns]
    upper=q2[columns]+1.5*iqr[columns]
    outliers=(test_new[columns]< lower)|(test_new[columns] > upper)
    return outliers
outlier_dict={i:identify(i) for i in test_new.columns}
outlier_list={}
for name,value in outlier_dict.items():
     if value.sum()>100:
        outlier_list[name]=value.sum()
outlier_list
for columns in test_new.columns:
    print(f"Before:")
    print(test_new[columns].describe())
    if columns in outlier_list:
        win_data=winsorize(test[columns],limits=[0.05,0.05])
        test_new[columns]=win_data
    print(f"After:")
    print(test[columns].describe())                       

Before:
count    45098.000000
mean         7.277507
std          2.591436
min          0.000000
25%          7.277507
50%          7.277507
75%          7.277507
max         15.000000
Name: county, dtype: float64
After:
count    45098.000000
mean         7.277507
std          2.591436
min          0.000000
25%          7.277507
50%          7.277507
75%          7.277507
max         15.000000
Name: county, dtype: float64
Before:
count    45098.000000
mean         0.553759
std          0.264235
min          0.000000
25%          0.553759
50%          0.553759
75%          0.553759
max          1.000000
Name: is_business, dtype: float64
After:
count    45098.000000
mean         0.553759
std          0.264235
min          0.000000
25%          0.553759
50%          0.553759
75%          0.553759
max          1.000000
Name: is_business, dtype: float64
Before:
count    45098.000000
mean         1.876942
std          0.586017
min          0.000000
25%          1.876942
50%          1.876942


In [89]:
attributes_test=[columns for columns in test_new if columns!="target" and columns!="data_block_id" and columns!="row_id" and columns!="prediction_unit_id"]
selected_attributes_test=[name for name,value in zip(attributes_test,mutual_scores) if value>0.5]
model_test_x=test_new[selected_attributes_test].values
y_pred_test=Model_xgb.predict(model_test_x)

In [90]:
y_pred_test

array([ 41.761356, 103.27012 ,  18.674562, ..., 290.74487 , 290.74487 ,
       290.74487 ], dtype=float32)

In [86]:
y_test_rand=Model.predict(model_test_x)
y_test_rand

array([ 34.21224667,  95.72903948,   8.49517615, ..., 274.8555601 ,
       274.8555601 , 274.8555601 ])

In [72]:
row_id=test['row_id'].tolist()
data_block_id=test['data_block_id'].tolist()
row_id=np.array(row_id)
data_block_id=np.array(data_block_id)
target=y_test_rand
file_n=(r"C:\Users\ArtisusXiren\Desktop\predict-energy-behavior-of-prosumers\Data\Sample_submission.csv")
with open(file_n,'w',newline='') as file:
    csv_w=csv.writer(file)
    csv_w.writerow(['row_id','data_block_id','target'])
    for row_id_val, data_block_id_val, target_val in zip(row_id,data_block_id,target):
        csv_w.writerow([row_id_val, data_block_id_val, target_val ])